# Profiling Code Execution with cProfile


In [122]:
import cProfile, pstats, io
from pstats import SortKey
import pandas as pd

from importlib.util import spec_from_loader, module_from_spec
from importlib.machinery import SourceFileLoader

# Custom import of local file "decode"
spec = spec_from_loader("decode", SourceFileLoader("decode", "../../decode"))
decode = module_from_spec(spec)
spec.loader.exec_module(decode)

# Custom import of local file "encode"
spec = spec_from_loader("encode", SourceFileLoader("encode", "../../encode"))
encode = module_from_spec(spec)
spec.loader.exec_module(encode)

In [123]:
def print_full(data_frame):
    """This function will display all rows of the dataframe.

    Args:
        data_frame (DataFrame): This is a pandas dataframe of
                                arbitrary length.
    """
    pd.set_option("display.max_rows", len(data_frame))
    pd.set_option("display.width", 1000)
    print(data_frame)
    pd.reset_option("display.max_rows")
    pd.reset_option("display.width")

In [154]:
file = "../../data/0ab237b7-fb12-4687-afed-8d1e2070d621.wav"
compressed_file_path = "../../data/0ab237b7-fb12-4687-afed-8d1e2070d621.wav.brainwire"
decompressed_file_path = "../../data/0ab237b7-fb12-4687-afed-8d1e2070d621.wav.copy"

In [155]:
parser = encode.initialize_argument_parser()
args = parser.parse_args([file, compressed_file_path, "-m=u", "-v"])
encode.main(args)

decode_parser = decode.initialize_argument_parser()
decode_args = decode_parser.parse_args(
    [compressed_file_path, decompressed_file_path, "-v"]
)
decode.main(decode_args)


Executed Line: encode_using_amplitude_indices...
Time Δ Nanoseconds: 166749000
Time Δ Microseconds: 166749.0
Time Δ Milliseconds: 166.749
Time Δ Seconds: 0.166749


Original File Size: 197526
Compressed File Size: 74054
Percent of Compression: 62.51%
Percent of Compressed File Size Relative to Required File Size: 7502.938%

Executed Line: process_huffman_encoded_amplitude_indices...
Time Δ Nanoseconds: 2055379000
Time Δ Microseconds: 2055379.0
Time Δ Milliseconds: 2055.379
Time Δ Seconds: 2.055379


Original File Size: 197526
Compressed File Size: 74054
Percent of Compression: 62.51%
Percent of Compressed File Size Relative to Required File Size: 7502.938%



Executed Line: process_huffman_encoded_file...
Time Δ Nanoseconds: 3663091000
Time Δ Microseconds: 3663091.0
Time Δ Milliseconds: 3663.091
Time Δ Seconds: 3.663091


Original File Size: 197526
Compressed File Size: 119487
Percent of Compression: 39.51%
Percent of Compressed File Size Relative to Required File Size: 12106.079%


In [126]:
pr = cProfile.Profile()
pr.enable()
encode.main(args)
s = io.StringIO()
sortby = SortKey.TIME
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print(s.getvalue())


Executed Line: encode_using_amplitude_indices...
Time Δ Nanoseconds: 804711000
Time Δ Microseconds: 804711.0
Time Δ Milliseconds: 804.711
Time Δ Seconds: 0.804711


Original File Size: 197526
Compressed File Size: 74054
Percent of Compression: 62.51%
Percent of Compressed File Size Relative to Required File Size: 7502.938%
         180949 function calls (180699 primitive calls) in 0.805 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.563    0.563    0.563    0.563 ../../encode:96(convertHexToBit)
        1    0.093    0.093    0.098    0.098 ../../encode:127(create_byte_string)
    98741    0.074    0.000    0.074    0.000 {method 'index' of 'list' objects}
        1    0.024    0.024    0.098    0.098 ../../encode:475(<listcomp>)
        1    0.019    0.019    0.019    0.019 ../../encode:66(determine_hex_freq)
        2    0.011    0.005    0.011    0.005 {built-in method io.open}
        2    0.005    0.00

In [91]:
s.getvalue()

"         123863 function calls (123735 primitive calls) in 2.398 seconds\n\n   Ordered by: cumulative time\n\n   ncalls  tottime  percall  cumtime  percall filename:lineno(function)\n        4    0.000    0.000    2.398    0.600 /Volumes/T7/Github/Neuralink-Compression-Challenge/py_encoding_env/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3541(run_code)\n        4    0.000    0.000    2.398    0.600 {built-in method builtins.exec}\n        1    0.000    0.000    2.398    2.398 ../../encode:667(main)\n        1    0.000    0.000    2.393    2.393 ../../encode:377(create_huffman_encoded_file)\n        1    0.001    0.001    1.951    1.951 ../../encode:317(huffman_encoding)\n        1    1.896    1.896    1.896    1.896 ../../encode:96(convertHexToBit)\n        1    0.435    0.435    0.441    0.441 ../../encode:127(create_byte_string)\n        1    0.054    0.054    0.055    0.055 ../../encode:66(determine_hex_freq)\n   118746    0.005    0.000    0.005    0.000 {method 

In [75]:
with cProfile.Profile() as pr:

    encode.main(args)

df = pd.DataFrame(
    pr.getstats(),
    columns=["func", "ncalls", "ccalls", "tottime", "cumtime", "callers"],
)


Executed Line: create_huffman_encoded_file...
Time Δ Nanoseconds: 2433533000
Time Δ Microseconds: 2433533.0
Time Δ Milliseconds: 2433.533
Time Δ Seconds: 2.433533


Original File Size: 197526
Compressed File Size: 119487
Percent of Compression: 39.51%
Percent of Compressed File Size Relative to Required File Size: 12106.079%


In [68]:
df[df["cumtime"] == df["cumtime"].max()]

,func,ncalls,ccalls,tottime,cumtime,callers
160,"<code object convertHexToBit at 0x15aeb4200, f...",1,0,1.914256,1.914092,"[(<built-in method builtins.len>, 3, 0, 3.34e-..."


In [76]:
df.sort_values("tottime", ascending=False)

,func,ncalls,ccalls,tottime,cumtime,callers
158,"<code object main at 0x12f46da00, file ""../../...",1,0,2.433800e+00,2.862600e-05,"[(<built-in method time.time_ns>, 2, 0, 1.624e..."
164,<code object create_huffman_encoded_file at 0x...,1,0,2.431411e+00,2.904200e-05,[(<code object create_byte_string at 0x10da0b4...
122,"<code object huffman_encoding at 0x10da0c200, ...",1,0,1.979189e+00,5.039550e-04,"[(<built-in method builtins.len>, 66, 0, 4.625..."
160,"<code object convertHexToBit at 0x15aeb4200, f...",1,0,1.921389e+00,1.921221e+00,"[(<built-in method builtins.len>, 3, 0, 1.001e..."
121,<code object create_byte_string at 0x10da0b400...,1,0,4.521933e-01,4.463963e-01,"[(<method 'append' of 'list' objects>, 6, 0, 7..."
...,...,...,...,...,...,...
4,<method 'reverse' of 'list' objects>,3,0,2.500000e-07,2.500000e-07,None
21,<method 'rstrip' of 'str' objects>,1,0,2.090000e-07,2.090000e-07,None
26,<built-in method builtins.iter>,1,0,1.670000e-07,1.670000e-07,None
37,<built-in method posix.fspath>,1,0,1.670000e-07,1.670000e-07,None


In [56]:
df.iloc(1)

In [57]:
print_full(df.sort_values("tottime", ascending=False))

                                                  func  ncalls  ccalls       tottime       cumtime                                            callers
158  <code object main at 0x12f46da00, file "../../...       1       0  2.378402e+00  5.233300e-05  [(<code object print_size_of_file_compression ...
164  <code object create_huffman_encoded_file at 0x...       1       0  2.368149e+00  4.370800e-05  [(<code object create_byte_string at 0x10da0b4...
122  <code object huffman_encoding at 0x10da0c200, ...       1       0  1.918681e+00  5.630850e-04  [(<code object convertHexToBit at 0x15aeb4200,...
160  <code object convertHexToBit at 0x15aeb4200, f...       1       0  1.877472e+00  1.877314e+00  [(<method 'hex' of 'bytes' objects>, 1, 0, 0.0...
121  <code object create_byte_string at 0x10da0b400...       1       0  4.494243e-01  4.435484e-01  [(<method 'append' of 'list' objects>, 6, 0, 7...
157  <code object determine_hex_freq at 0x12f46bcc0...       1       0  3.993721e-02  3.959979e-02  